In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import matplotlib.pyplot as plt

# <span style="color:#FFCC80"> Swiss in vivo processing </span> </br>
Compare and analyze if using optimized parameters improves or deteriorates the final chi maps </br>

Keywords: </br>
* sw = slicewise
* pp = pipeline

In [2]:
# Working on MAC computer:
# Load metric data
sw_def_pp_gm_PDF_iLSQR = pd.read_csv(r"/Users/soothsayer7/msc_rios/msc_data_mac_processing/may_2025_mac_processing/BGFR_to_Chimap/def_PDF/chimap_def_iLSQR/chi_slicewise_mask_gm.csv")
sw_def_pp_wm_PDF_iLSQR = pd.read_csv(r"/Users/soothsayer7/msc_rios/msc_data_mac_processing/may_2025_mac_processing/BGFR_to_Chimap/def_PDF/chimap_def_iLSQR/chi_slicewise_mask_wm.csv")

sw_opt_pp_gm_PDF_iLSQR = pd.read_csv(r"/Users/soothsayer7/msc_rios/msc_data_mac_processing/may_2025_mac_processing/BGFR_to_Chimap/opt_PDF/Chimap_opt_iLSQR/chi_slicewise_mask_gm.csv")
sw_opt_pp_wm_PDF_iLSQR = pd.read_csv(r"/Users/soothsayer7/msc_rios/msc_data_mac_processing/may_2025_mac_processing/BGFR_to_Chimap/opt_PDF/Chimap_opt_iLSQR/chi_slicewise_mask_wm.csv")


In [3]:
print(sw_opt_pp_wm_PDF_iLSQR.columns)

Index(['Timestamp', 'SCT Version', 'Filename', 'Slice (I->S)', 'VertLevel',
       'DistancePMJ', 'Label', 'Size [vox]', 'WA()', 'STD()'],
      dtype='object')


# <span style="color:#CE93D8"> Spinal Cord Gray matter to White matter ratio </span> </br>

In [12]:
sw_def_pp_gm_score = sw_def_pp_gm_PDF_iLSQR.rename(columns={"Slice (I->S)": "Slice", "WA()": "GM_WA", "STD()": "GM_STD"})
sw_def_pp_wm_score = sw_def_pp_wm_PDF_iLSQR.rename(columns={"Slice (I->S)": "Slice", "WA()": "WM_WA", "STD()": "WM_STD"})

In [23]:
score_merged = pd.merge(sw_def_pp_gm_score, sw_def_pp_wm_score, on="Slice")

# Compute separability score
score_merged["GM_WM_Score"] = np.abs(score_merged["GM_WA"] - score_merged["WM_WA"]) / np.sqrt(score_merged["GM_STD"]**2 + score_merged["WM_STD"]**2)

# Sort and view
score_merged_sorted = score_merged.sort_values("GM_WM_Score", ascending=False)
print(score_merged_sorted[["Slice", "GM_WA", "WM_WA", "GM_STD", "WM_STD", "GM_WM_Score"]].head(3))

    Slice     GM_WA     WM_WA    GM_STD    WM_STD  GM_WM_Score
12     12  0.006628 -0.004349  0.006272  0.009859     0.939340
15     15  0.001795 -0.003321  0.003534  0.005764     0.756739
14     14  0.002628 -0.004732  0.007296  0.010307     0.582841


In [17]:
sw_opt_pp_gm_score = sw_opt_pp_gm_PDF_iLSQR.rename(columns={"Slice (I->S)": "Slice", "WA()": "GM_WA", "STD()": "GM_STD"})
sw_opt_pp_wm_score = sw_opt_pp_wm_PDF_iLSQR.rename(columns={"Slice (I->S)": "Slice", "WA()": "WM_WA", "STD()": "WM_STD"})

In [22]:
score_opt_merged = pd.merge(sw_opt_pp_gm_score, sw_opt_pp_wm_score, on="Slice")

# Compute separability score
score_opt_merged["GM_WM_Score"] = np.abs(score_opt_merged["GM_WA"] - score_opt_merged["WM_WA"]) / np.sqrt(score_opt_merged["GM_STD"]**2 + score_opt_merged["WM_STD"]**2)

# Sort and view
score_opt_merged_sorted = score_opt_merged.sort_values("GM_WM_Score", ascending=False)
print(score_opt_merged_sorted[["Slice", "GM_WA", "WM_WA", "GM_STD", "WM_STD", "GM_WM_Score"]].head(3))

    Slice     GM_WA     WM_WA    GM_STD    WM_STD  GM_WM_Score
12     12  0.007959 -0.006987  0.009289  0.014287     0.877017
8       8  0.007478 -0.004834  0.008513  0.018349     0.608656
14     14  0.001657 -0.007072  0.008364  0.013150     0.560122


## <span style="color:#FF5722"> Slicewise metrics </span> </br>

In [10]:
gm_df = sw_opt_pp_gm_PDF_iLSQR.rename(columns={"Slice (I->S)": "Slice", "WA()": "WA"})
wm_df = sw_opt_pp_wm_PDF_iLSQR.rename(columns={"Slice (I->S)": "Slice", "WA()": "WA"})

# Add tissue labels
gm_df["Tissue"] = "GM"
wm_df["Tissue"] = "WM"

# Combine both DataFrames
violin_df = pd.concat([gm_df[["Slice", "WA", "Tissue"]],
                       wm_df[["Slice", "WA", "Tissue"]]])

# Plot violin
plt.figure(figsize=(14, 6))
sns.violinplot(x="Slice", y="WA", hue="Tissue", data=violin_df, split=True)

plt.title("Slice-wise Violin Plot of GM and WM Susceptibility (PDF-iLSQR)")
plt.xlabel("Slice (I→S)")
plt.ylabel("WA (Susceptibility)")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()